In [1]:
# !git clone https://github.com/milki93/Amharic-E-commerce-Data-Extractor.git

Cloning into 'Amharic-E-commerce-Data-Extractor'...
remote: Enumerating objects: 18862, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 18862 (delta 33), reused 40 (delta 15), pack-reused 18795 (from 1)
Receiving objects: 100% (18862/18862), 106.71 MiB | 10.69 MiB/s, done.
Resolving deltas: 100% (2019/2019), done.
Updating files: 100% (18558/18558), done.


In [2]:
%cd /content/Amharic-E-commerce-Data-Extractor/notebooks

/content/Amharic-E-commerce-Data-Extractor/notebooks


In [5]:
# !pip install torch transformers shap lime datasets

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install torch

### Load the conll file and the best model

In [8]:
import os
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
import numpy as np

conll_file_path = "/content/Amharic-E-commerce-Data-Extractor/data/labeled_data.conll"
best_model_path = "/content/drive/MyDrive/my_ner_models/bert_base_multilingual_cased/checkpoint-30"

def load_conll_data(file_path):
    sentences = []
    current_words = []
    current_labels = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split()
                if len(parts) == 2:
                    word, label = parts
                    current_words.append(word)
                    current_labels.append(label)
                else:
                    print(f"Skipping malformed line in CoNLL: '{line}'")
            else:
                if current_words:
                    sentences.append({"words": current_words, "labels": current_labels})
                current_words = []
                current_labels = []

    if current_words:
        sentences.append({"words": current_words, "labels": current_labels})
    return sentences

try:
    labeled_data = load_conll_data(conll_file_path)
    print(f"Successfully loaded {len(labeled_data)} sentences from {conll_file_path}")
    if labeled_data:
        print("First 2 loaded sentences:")
        for i, sentence in enumerate(labeled_data[:2]):
            print(f"Sentence {i+1}: {' '.join(sentence['words'])}")
            print(f"Labels   {i+1}: {sentence['labels']}")
except FileNotFoundError:
    print(f"Error: CoNLL file not found at {conll_file_path}. Please ensure the path is correct and the file exists.")
    labeled_data = []

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained(best_model_path)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

id2label = model.config.id2label
label2id = model.config.label2id
labels = [id2label[i] for i in sorted(id2label.keys())]

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1, aggregation_strategy="simple")

print("\nModel and Tokenizer loaded.")
print("Detected Labels:", labels)
lime_class_names = labels

Successfully loaded 50 sentences from /content/Amharic-E-commerce-Data-Extractor/data/labeled_data.conll
First 2 loaded sentences:
Sentence 1: skechers archfit size 40 41 42 43 price 3400 birr አድራሻ - ሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ 101 የቢሮ ቁጥር ያገኙናል or call 0920238243 ethiobrand
Labels   1: ['B-Product', 'B-Product', 'O', 'O', 'O', 'O', 'O', 'B-PRICE', 'I-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O']
Sentence 2: magic silicone dish washing gloves high quality ወፍራሙ ዕቃ ለማጠብ ቤት ለማፅዳት መኪና ለማጠብ ለተለያየ አገልግሎት የሚውል ዋጋ፦ 350ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ s05s06 0902660722 0928460606 በtelegram ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን
Labels   2: ['O', 'B-Product', 'I-Product', 'I-Product', 'B-Product', 'O', 'O', 'B-Product', 'I-Product', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PRICE', 'I-PRICE', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LO

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Device set to use cpu



Model and Tokenizer loaded.
Detected Labels: ['O', 'B-LOC', 'B-PRICE', 'B-Product', 'I-LOC', 'I-PRICE', 'I-Product']


In [15]:
texts_for_interpretation = []
for entry in labeled_data:
    texts_for_interpretation.append(" ".join(entry["words"]))

if len(texts_for_interpretation) > 0:
    num_examples_to_interpret = min(5, len(texts_for_interpretation))
    selected_texts = texts_for_interpretation[:num_examples_to_interpret]
    selected_ground_truth_labels = [labeled_data[i]["labels"] for i in range(num_examples_to_interpret)]

    print("\nNER Pipeline Output for Example Texts:")
    for i, text in enumerate(selected_texts):
        print(f"Text {i+1}: {text}")
        predictions = ner_pipeline(text)
        if predictions:
            for entity in predictions:
                print(f"  - Entity: {entity['word']} | Type: {entity['entity_group']} | Score: {entity['score']:.2f}")
        else:
            print("  - No entities found.")
        print("." * 30)

else:
    print("No labeled data loaded. Cannot proceed with selecting examples for interpretation.")
    selected_texts = []
    selected_ground_truth_labels = []


NER Pipeline Output for Example Texts:
Text 1: skechers archfit size 40 41 42 43 price 3400 birr አድራሻ - ሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ 101 የቢሮ ቁጥር ያገኙናል or call 0920238243 ethiobrand
  - Entity: skechers archfit | Type: Product | Score: 0.40
  - Entity: 340 | Type: PRICE | Score: 0.38
  - Entity: birr | Type: PRICE | Score: 0.42
  - Entity: አድራሻ - ሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ 101 የቢሮ ቁጥር ያገኙናል | Type: LOC | Score: 0.92
..............................
Text 2: magic silicone dish washing gloves high quality ወፍራሙ ዕቃ ለማጠብ ቤት ለማፅዳት መኪና ለማጠብ ለተለያየ አገልግሎት የሚውል ዋጋ፦ 350ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ s05s06 0902660722 0928460606 በtelegram ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን
  - Entity: magic silicone dish washing gloves high quality | Type: Product | Score: 0.65
  - Entity: ያለው አድራሻ መገናኛ _ መሰረት _ ደፋር _ ሞል _ ሁለተኛ _ ፎቅ ቢሮ ቁ s05 | Type: LOC | Score: 0.56
  - Entity: ##0 | Type: LOC | Score: 0.36
..............................
Text 3: nike just do it size 39 40 4

### LIME and SHAP Prediction Functions

In [17]:
def lime_predict_proba(texts):
    results = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits

        probabilities = torch.softmax(logits, dim=-1)
        averaged_probs = torch.mean(probabilities, dim=1).squeeze().cpu().numpy()
        results.append(averaged_probs)
    return np.array(results)

def shap_predict_sentence_tag_probs(texts):
    all_sentence_probs = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        probabilities = torch.softmax(logits, dim=-1)
        averaged_probs = torch.mean(probabilities, dim=1).squeeze().cpu().numpy()
        all_sentence_probs.append(averaged_probs)
    return np.array(all_sentence_probs)

### Lime interpretation

In [18]:
import lime
import shap
from lime.lime_text import LimeTextExplainer

print("\nApplying LIME for interpretation on selected CoNLL data examples")

explainer = LimeTextExplainer(class_names=lime_class_names)
b_product_idx = lime_class_names.index('B-Product')
b_price_idx = lime_class_names.index('B-PRICE')
b_loc_idx = lime_class_names.index('B-LOC')

for i, text_to_explain in enumerate(selected_texts):
    print(f"\n Explaining Text {i+1}: '{text_to_explain}'")
    print(f"  Ground Truth Labels: {selected_ground_truth_labels[i]}")

    model_predictions = ner_pipeline(text_to_explain)
    print("  Model Predicted Entities:")
    for entity in model_predictions:
        print(f"    - {entity['word']} ({entity['entity_group']}) Score: {entity['score']:.2f}")

    explanation = explainer.explain_instance(
        text_to_explain,
        lime_predict_proba,
        num_features=5,
        num_samples=500,
        top_labels=None
    )

    target_indices_to_explain = [b_product_idx, b_price_idx, b_loc_idx]
    lime_output_dir = "lime_explanations"
    os.makedirs(lime_output_dir, exist_ok=True)

    for label_idx in target_indices_to_explain:
        label_name = lime_class_names[label_idx]
        if label_idx in explanation.local_exp:
            print(f"\n LIME Explanation for {label_name}")
            print(f"  Words contributing to {label_name} prediction:")
            for word, weight in explanation.as_list(label=label_idx):
                print(f"    {word}: {weight:.4f}")
            file_name = f'lime_explanation_text_{i+1}_label_{label_name}.html'
            file_path = os.path.join(lime_output_dir, file_name)
            explanation.save_to_file(file_path, labels=[label_idx])
            print(f"    LIME explanation saved to: {file_path}")
        else:
            print(f"\n     No LIME explanation generated for {label_name} for this text.")
    print("\n" + "."*80)



Applying LIME for interpretation on selected CoNLL data examples

 Explaining Text 1: 'skechers archfit size 40 41 42 43 price 3400 birr አድራሻ - ሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ 101 የቢሮ ቁጥር ያገኙናል or call 0920238243 ethiobrand'
  Ground Truth Labels: ['B-Product', 'B-Product', 'O', 'O', 'O', 'O', 'O', 'B-PRICE', 'I-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O']
  Model Predicted Entities:
    - skechers archfit (Product) Score: 0.40
    - 340 (PRICE) Score: 0.38
    - birr (PRICE) Score: 0.42
    - አድራሻ - ሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ 101 የቢሮ ቁጥር ያገኙናል (LOC) Score: 0.92

     No LIME explanation generated for B-Product for this text.

     No LIME explanation generated for B-PRICE for this text.

 LIME Explanation for B-LOC
  Words contributing to B-LOC prediction:
    0920238243: -0.0076
    size: -0.0048
    price: -0.0036
    birr: 0.0035
    or: -0.002

In [19]:
print("\nApplying SHAP for interpretation on selected CoNLL data")

if len(selected_texts) > 0:
    masker = shap.maskers.Text(tokenizer, mask_token=tokenizer.mask_token)
    explainer_shap = shap.Explainer(shap_predict_sentence_tag_probs, masker)

    for i, text_to_explain in enumerate(selected_texts):
        print(f"\n SHAP Explanation for Text {i+1}: '{text_to_explain}'")
        print(f"  Ground Truth Labels: {selected_ground_truth_labels[i]}")

        shap_values = explainer_shap(
            [text_to_explain],
            outputs=shap.Explanation.argsort.flip[:3]
        )

        print("\n  SHAP values (explaining sentence-level entity presence for top 3 labels):")

        for output_idx in range(len(shap_values[0].output_names)):
            label_name = shap_values[0].output_names[output_idx]
            print(f"\n Explaining '{label_name}'")
            base_value = shap_values[0].base_values[output_idx]
            feature_values = shap_values[0].values[output_idx]
            feature_data = shap_values[0].data

            print(f"    Base value (average prediction for '{label_name}'): {base_value:.4f}")
            print(f"    Word contributions for '{label_name}':")
            word_contributions = list(zip(feature_data, feature_values))
            word_contributions.sort(key=lambda x: abs(x[1]), reverse=True)
            for word, weight in word_contributions:
                print(f"      {word}: {weight:.4f}")
        print("\n" + "."*80)

else:
    print("No selected texts for SHAP analysis.")


Applying SHAP for interpretation on selected CoNLL data

 SHAP Explanation for Text 1: 'skechers archfit size 40 41 42 43 price 3400 birr አድራሻ - ሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ 101 የቢሮ ቁጥር ያገኙናል or call 0920238243 ethiobrand'
  Ground Truth Labels: ['B-Product', 'B-Product', 'O', 'O', 'O', 'O', 'O', 'B-PRICE', 'I-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O']


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [01:11, 71.92s/it]               



  SHAP values (explaining sentence-level entity presence for top 3 labels):

 Explaining 'Output 0'
    Base value (average prediction for 'Output 0'): 0.3768
    Word contributions for 'Output 0':
      : 0.0000
      sk: 0.0000
      eche: 0.0000

 Explaining 'Output 1'
    Base value (average prediction for 'Output 1'): 0.1282
    Word contributions for 'Output 1':
      : -0.0199
      eche: 0.0125
      sk: -0.0014

 Explaining 'Output 2'
    Base value (average prediction for 'Output 2'): 0.1580
    Word contributions for 'Output 2':
      eche: 0.0211
      : -0.0197
      sk: -0.0067

................................................................................

 SHAP Explanation for Text 2: 'magic silicone dish washing gloves high quality ወፍራሙ ዕቃ ለማጠብ ቤት ለማፅዳት መኪና ለማጠብ ለተለያየ አገልግሎት የሚውል ዋጋ፦ 350ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ s05s06 0902660722 0928460606 በtelegram ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን'
  Ground Truth Labels: ['O', 'B-Product', 'I-Product', '

  0%|          | 0/498 [00:00<?, ?it/s]

...

PartitionExplainer explainer: 2it [01:00, 60.10s/it]               



  SHAP values (explaining sentence-level entity presence for top 3 labels):

 Explaining 'Output 0'
    Base value (average prediction for 'Output 0'): 0.3768
    Word contributions for 'Output 0':
      jord: 0.0086
      : -0.0050
      air : 0.0021

 Explaining 'Output 1'
    Base value (average prediction for 'Output 1'): 0.1282
    Word contributions for 'Output 1':
      jord: 0.0125
      : -0.0057
      air : 0.0018

 Explaining 'Output 2'
    Base value (average prediction for 'Output 2'): 0.1580
    Word contributions for 'Output 2':
      : -0.0101
      jord: 0.0072
      air : -0.0008

................................................................................

 SHAP Explanation for Text 5: 'በረፍት ቀንዎ ሱቅ ላይ መስተናገድ ለምትፈልጉ ውድ ደንበኞቻችን ነገ ከጠዋቱ 430 _ ቀኑ 1100 ድረስ ሱቃችን ክፍት ሁኖ ይጠብቅዎታል ሱቅ መተው መግዛት ላልቻላችሁ በሞተረኞች ያሉበት እናደርሳለን ዘወትር ሰኞ _ቅዳሜ ከጠዋቱ 230 እስከ ምሽቱ 200 ድረስ ክፍት መሆኑን እንገልፃለን አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ s05s06 0902660722 0928460606 በtelegram ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግ

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [01:11, 71.14s/it]               


  SHAP values (explaining sentence-level entity presence for top 3 labels):

 Explaining 'Output 0'
    Base value (average prediction for 'Output 0'): 0.3768
    Word contributions for 'Output 0':
      : 0.0048
      በረፍት : -0.0019
      ቀንዎ : -0.0008

 Explaining 'Output 1'
    Base value (average prediction for 'Output 1'): 0.1282
    Word contributions for 'Output 1':
      : 0.0089
      ቀንዎ : -0.0028
      በረፍት : -0.0007

 Explaining 'Output 2'
    Base value (average prediction for 'Output 2'): 0.1580
    Word contributions for 'Output 2':
      : 0.0074
      ቀንዎ : -0.0022
      በረፍት : -0.0003

................................................................................


#### LIME & SHAP Insights

**Location (B-LOC):**
- **LIME** correctly highlighted location words like "መገናኛ..." with positive contributions.
- Some expected words like "አድራሻ" or phone numbers had low or negative impact, suggesting inconsistent model understanding.
- **SHAP** also highlighted useful words, but it used generic labels like "Output 0", making it hard to match them to B-LOC directly.

**Product (B-Product) & Price (B-PRICE):**
- **LIME** gave no explanations for these, meaning the model didn't predict them confidently.
- Predictions like "skechers archfit" (0.40) and "340" (0.38) had lower scores compared to B-LOC predictions.
- **SHAP** couldn't give clear insights either due to missing label mapping.

---

####  Areas for Improvement

1. **Lack of Product & Price Examples:**
   - The model struggles with these entities.
   - Add more labeled examples with diverse products and price formats.

2. **Unclear SHAP Output:**
   - SHAP uses generic output names.
   - Map SHAP outputs to actual labels (B-LOC, B-Product, etc.) using id2label.

3. **Tokenization Confusion:**
   - Some relevant words are not contributing as expected.
   - Review how compound Amharic words and numbers are tokenized. Add clearer examples.


The model performs well on **Location** entities but struggles with **Product** and **Price**. Improving training data and label mapping in SHAP will boost both performance and interpretability.


In [ ]:
import nbformat
notebook_path = "/content/Amharic-E-commerce-Data-Extractor/notebooks/model_Interpretability.ipynb"
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']
    print("Removed metadata.widgets")

with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)
print("Notebook cleaned and saved.")

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/model_Interpretability.ipynb" /content/Amharic-E-commerce-Data-Extractor/notebooks

In [ ]:
%cd /content/Amharic-E-commerce-Data-Extractor

In [ ]:
# !ssh-keygen -t ed25519 -C "milkidida131@gmail.com" -f ~/.ssh/id_ed25519 -N ""

In [ ]:
# !mkdir -p ~/.ssh
# !eval "$(ssh-agent -s)" && ssh-add ~/.ssh/id_ed25519

In [ ]:
# !mkdir -p ~/.ssh && ssh-keyscan github.com >> ~/.ssh/known_hosts

In [ ]:
# !cat ~/.ssh/id_ed25519.pub

In [ ]:
# !ssh -T git@github.com

In [ ]:
!git remote set-url origin git@github.com:milki93/Amharic-E-commerce-Data-Extractor.git

In [ ]:
!git checkout -b task-5

In [ ]:
!git add notebooks/model_comparison_ner.ipynb
!git commit -m "Add model comparison of NER appling on the dataset"
!git push origin task-4